In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

In [8]:
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        return x
    

class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
   
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [17]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the input and output sizes
input_size = 784
hidden_size = 256
output_size = 1

# Create the discriminator and generator
discriminator = Discriminator(input_size, hidden_size, output_size).to(device)
generator = Generator(input_size, hidden_size, output_size).to(device)

# Set the loss function and optimizers
loss_fn = nn.BCEWithLogitsLoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002)

# Set the number of epochs and the noise size
num_epochs = 5
noise_size = 784

# MNIST Dataset transformation - Converts to tensor and normalizes the images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1] range
])

# Downloading and loading the MNIST dataset
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(mnist_train, batch_size=64, shuffle=True)

# Training loop
for epoch in range(num_epochs):
  for i, (real_images, _) in enumerate(dataloader):
    # Get the batch size
    real_images = real_images.cuda()
    
    batch_size = real_images.size(0)
    print(batch_size)
    break
  break

64


In [18]:
 # Generate fake images
noise = torch.randn(batch_size, noise_size).to(device)
fake_images = generator(noise)

# Train the discriminator on real and fake images
d_real = discriminator(real_images)
d_fake = discriminator(fake_images)

# Calculate the loss
real_loss = loss_fn(d_real, torch.ones_like(d_real))
fake_loss = loss_fn(d_fake, torch.zeros_like(d_fake))
d_loss = real_loss + fake_loss

# Backpropagate and optimize
d_optimizer.zero_grad()
d_loss.backward()
d_optimizer.step()

# Train the generator
d_fake = discriminator(fake_images)
g_loss = loss_fn(d_fake, torch.ones_like(d_fake))

# Backpropagate and optimize
g_optimizer.zero_grad()
g_loss.backward()
g_optimizer.step()

# Print the loss every 50 batches
if (i+1) % 50 == 0:
        print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}'
              .format(epoch+1, num_epochs, i+1, len(dataloader), d_loss.item(), g_loss.item()))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1792x28 and 784x256)